In [2]:
import requests
import json
import pandas as pd
import requests
import pprint
import pickle
import os 
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent

In [3]:
def headers(token):
    """
    Headers necesarios para autenticarse en la API de Twitter.
    """
    bearer_token = token
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def tweets_covid(query, headers, max_results = 10):
    """
    Se hace la petición.
    
    max_results solo puede ser un número entre 10 y 100. Por default la resuesta son 10 tweets.
    """
    #endpoint_URL = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results={}&expansions=geo.place_id&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type&tweet.fields=geo".format(query, max_results)
    #endpoint_URL = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results={}&expansions=geo.place_id&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type&tweet.fields=geo,geo.coordinates,referenced_tweets,source".format(query, max_results)
    #endpoint_URL = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results={}&expansions=geo.place_id&tweet.fields=geo&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type".format(query, max_results)
    endpoint_URL = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results={}&user.fields=location&expansions=geo.place_id,author_id&tweet.fields=geo,created_at,public_metrics&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type".format(query, max_results)
    response = requests.request("GET", endpoint_URL, headers = headers).json()
    return response


def create_dataframe(response):
    """
    Crear DataFrame
    """
    df = pd.DataFrame(response["data"])
    data_loc = response["includes"]
    df2 = pd.DataFrame(data_loc["users"])
    #fichero_binario = open("locations", "wb")
    #pickle.dump(pickle_chido, fichero_binario)
    #fichero_binario.close()
    return (df, df2)
    

def main():
    query = "covid"
    #token = "AAAAAAAAAAAAAAAAAAAAADO8PQEAAAAA55OlrttdRo30obn2t5qp4SMJQUY%3D4fLbfXQQ08e8dNVRs8xlRcmwnDXwaKl7zQvQ5ZcEyijjV3d4RI"
    token = os.environ["TOKEN_TT"].replace('"','')
    max_tweets = 100
    
    # Headers
    encabezados = headers(token)
    
    # Tweets
    tweets = tweets_covid(query, encabezados, max_tweets)
    #pprint.pprint(tweets)
    
    # Crear DataFame
    dfs = create_dataframe(tweets)
    df_covid_new = dfs[0]
    df_locations_new = dfs[1]
    # Save data
    #df_covid_new.to_csv("covid.csv")
    #df_locations_new.to_csv("tweets_locations.csv")

    #### Update covid csv ####
    ##### old #####
    df_covid_old = pd.read_csv("covid.csv")
    if "Unnamed: 0" in df_covid_old.columns.tolist():
        df_covid_old.drop(["Unnamed: 0"], axis = 1, inplace = True)
    elif "geo" in df_covid_old.columns.tolist():
        df_covid_old.drop(["geo"], axis = 1, inplace = True)
    # Reordering old data columns
    df_covid_old = df_covid_old[["author_id", "created_at", "public_metrics", "text", "id"]]
    
    if "geo" in df_covid_new.columns.tolist():
        df_covid_new.drop(["geo"], axis = 1, inplace = True)
    # Reordering new data columns 
    df_covid_new = df_covid_new[["author_id", "created_at", "public_metrics", "text", "id"]]
    
    # reset index
    df_covid_old.reset_index(drop=True)
    df_covid_new.reset_index(drop=True)

    # astype str
    df_covid_old = df_covid_old.applymap(str)
    df_covid_new = df_covid_new.applymap(str)

    #display(df_covid_old.columns.tolist())
    #display(df_covid_old.head(1))
    #display(df_covid_new.columns.tolist())
    #display(df_covid_new.head(1))

    # Dataframe difference
    new_rows = df_covid_new[~df_covid_new.apply(tuple,1).isin(df_covid_old.apply(tuple,1))]
    frames = [df_covid_old, new_rows]
    pd.concat(frames).to_csv("covid.csv", index = False)
    
    ##### update locations csv #####
    df_locations_old = pd.read_csv("tweets_locations.csv")
    if "Unnamed: 0" in df_locations_old.columns.tolist():
        df_locations_old.drop(["Unnamed: 0"], axis = 1, inplace = True)
    
    df_locations_old = df_locations_old[['id', 'location', 'username', 'name']]
    df_locations_new = df_locations_new[['id', 'location', 'username', 'name']]
    
    # reset index
    df_locations_old.reset_index(drop=True)
    df_locations_new.reset_index(drop=True)

    # astype str
    df_locations_old = df_locations_old.applymap(str)
    df_locations_new = df_locations_new.applymap(str)

    # concat dataframes then save them into csv file
    pd.concat([df_locations_old, df_locations_new]).to_csv("tweets_locations.csv", index = False)

if __name__ == "__main__":
    main()

In [4]:
df_covid = pd.read_csv("covid.csv")
#display(df_covid.sample())
print(df_covid.shape)

(1399, 5)


In [5]:
#df_covid[df_covid.duplicated()]

In [6]:
df_locations = pd.read_csv("tweets_locations.csv")
#display(df_locations.sample())
print(df_locations.shape)

(897, 4)


In [7]:
#df_locations.value_counts(subset=["location"])

In [8]:
df_locations

,id,location,username,name
0,1041202867594358785,chanbaek ‎(✿˘˘✿),byunfuse06,배큥 D-438
1,21118184,London & Liverpool,soapachu,badly-drawn bee 🐝
2,104075684,"Minnesota, USA",ThisIsNotTea,Thisisnottea
3,1197757502307360768,"Soweto, South Africa",Mmathuto_,MaMthethwa🦋
4,226098607,Thailand,baifern17589,♣︎ｂａｉｆｅｒｎ° feat. 공감🌹
...,...,...,...,...
892,1444009032738607107,NaN,qdrv8cqj9w,Julie Habberfield
893,1286624674957406209,NaN,AlwaysAuntB,AlwaysAuntB
894,39310815,The Mahogany Room,Blackmhgny,Blackmhgny
895,1113824938215997440,USA,thiborz,🇺🇲 Omicron Mogwai™🇵🇷


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a1e7defd-57f6-4a69-933f-51f8c5c266bb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>